# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [1]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

## Откройте и изучите файл

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбейте данные на выборки

In [6]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Разобьем данные на выборки в соотношении обучающая : валидационная : тестовая 60:20:20.

In [7]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size = 0.2, random_state=12345)

In [8]:
features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, test_size = 0.25, random_state=12345)

Проверим размеры получившихся выборок

In [9]:
print('Доля обучающей выборки {0:.2f}'.format(features_train.size / features.size))

Доля обучающей выборки 0.60


In [10]:
print('Доля валидационной выборки {0:.2f}'.format(features_valid.size / features.size))

Доля валидационной выборки 0.20


In [11]:
print('Доля тестовой выборки {0:.2f}'.format(features_test.size / features.size))

Доля тестовой выборки 0.20


## Исследуйте модели

### Дерево решений

Первую модель обучим алгоритмом дерева решений.

In [12]:
first_model = tree.DecisionTreeClassifier(random_state=12345)
first_model.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

Узнаем параметры получившегося дерева, чтобы отталкиваться от них в дальнейших исследованиях.

In [13]:
answer = first_model.predict(features_valid)
print('Доля правильных ответов', accuracy_score(target_valid, answer))

Доля правильных ответов 0.7309486780715396


In [14]:
est_max_depth = first_model.tree_.max_depth
print('Глубина дерева', est_max_depth)

Глубина дерева 28


Теперь обучим модели с разными гиперпараметрами, оценим качество каждой из них по доле правильных ответов и выберем лучшую модель.

In [15]:
first_accuracy = 0
for depth in range(1, est_max_depth + 1):
    for sample_n in range(2, 100, 5):
        for criterion in ['gini', 'entropy']:
            first_model = tree.DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split=sample_n, criterion=criterion)
            first_model.fit(features_train, target_train)
            answer = first_model.predict(features_valid)
            accuracy = accuracy_score(target_valid, answer)
            if accuracy > first_accuracy:
                first_accuracy = accuracy
                best_depth = depth
                best_first_model = first_model
                best_sample_n = sample_n
                best_criterion = criterion
print('Лучшая доля правильных ответов', first_accuracy)
print('Глубина дерева', best_depth)
print('Минимальное количество образцов для разделения внутреннего узла', best_sample_n)
print('Критерий оценки качества разделения', best_criterion)

Лучшая доля правильных ответов 0.8009331259720062
Глубина дерева 6
Минимальное количество образцов для разделения внутреннего узла 22
Критерий оценки качества разделения entropy


Попробуем также подобрать лучшие гиперпараметры с помощью поиска по сетке.

In [16]:
grid={"max_depth":range(1, est_max_depth+1), "min_samples_split":range(2, 100 ,5), "criterion":['gini', 'entropy']}
first_model=tree.DecisionTreeClassifier(random_state=12345)
first_model_cv=GridSearchCV(first_model,grid,cv=10, scoring = 'accuracy')
first_model_cv.fit(features_train,target_train)

print("tuned hyperparameters :(best parameters) ",first_model_cv.best_params_)
print("accuracy :",first_model_cv.best_score_)

tuned hyperparameters :(best parameters)  {'criterion': 'entropy', 'max_depth': 8, 'min_samples_split': 92}
accuracy : 0.8143153526970954


Результат получился немного другой и доля правильных ответов для такой модели выше.

### Случайный лес

Вторую модель будем обучать алгоритмом случайный лес.

In [17]:
second_model = RandomForestClassifier(random_state=12345)
second_model.fit(features_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [18]:
answer = second_model.predict(features_valid)
print('Доля правильных ответов', accuracy_score(target_valid, answer))

Доля правильных ответов 0.7869362363919129


Теперь обучим модели с разной максимальной глубиной и количеством деревьев, оценим качество каждой из них по доле правильных ответов и выберем лучшую модель.

In [19]:
second_accuracy = 0
for depth in range(1, est_max_depth + 1):
    for est in range(1, 101, 10):
        second_model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        second_model.fit(features_train, target_train)
        answer = second_model.predict(features_valid)
        accuracy = accuracy_score(target_valid, answer)
        if accuracy > second_accuracy:
            second_accuracy = accuracy
            best_depth = depth
            best_second_model = second_model
            best_n_est = est
print('Лучшая доля правильных ответов', second_accuracy)
print('Глубина дерева',best_depth)
print('Количество деревьев', best_n_est)

Лучшая доля правильных ответов 0.8009331259720062
Глубина дерева 9
Количество деревьев 81


Попробуем также подобрать лучшие гиперпараметры с помощью поиска по сетке.

In [20]:
grid={"max_depth":range(1, est_max_depth+1), "n_estimators":range(1, 101, 10)}
second_model=RandomForestClassifier(random_state=12345)
second_model_cv=GridSearchCV(second_model,grid,cv=10, scoring = 'accuracy')
second_model_cv.fit(features_train,target_train)

print("tuned hyperparameters :(best parameters) ",second_model_cv.best_params_)
print("accuracy :",second_model_cv.best_score_)

tuned hyperparameters :(best parameters)  {'max_depth': 8, 'n_estimators': 61}
accuracy : 0.8220954356846473


Поиск по сетке еще улучшил результат модели.

### Логистическая регрессия

Третью модель обучим с помощью алгоритма логистической регрессии.

In [21]:
third_model = LogisticRegression(random_state=12345)
third_model.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
answer = third_model.predict(features_valid)
third_accuracy = accuracy_score(target_valid, answer)
print('Доля правильных ответов', third_accuracy)

Доля правильных ответов 0.702954898911353


Попробуем увеличить долю правильных ответов нормализовав исходные данные.

In [23]:
scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(features_train)
features_train_norm = scaler.transform(features_train)
features_valid_norm = scaler.transform(features_valid)

In [24]:
third_model.fit(features_train_norm, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
answer = third_model.predict(features_valid_norm)
third_accuracy = accuracy_score(target_valid, answer)
print('Доля правильных ответов', third_accuracy)

Доля правильных ответов 0.7527216174183515


Доля правильных ответов увеличилась. Теперь попробуем вручную подобрать параметр С, посмотрим, как он влияет на качество модели.

In [26]:
C_param_range = [0.001,0.01,0.1,1,10,100]
for i in C_param_range:
    third_model = LogisticRegression(random_state=12345, C = i)
    third_model.fit(features_train_norm, target_train)
    answer = third_model.predict(features_valid_norm)
    accuracy = accuracy_score(target_valid, answer)
    if accuracy > third_accuracy:
        third_accuracy = accuracy
        best_c = i
        best_third_model = third_model
print('Лучшая доля правильных ответов', third_accuracy)
print('Лучший параметр С', best_c)

Лучшая доля правильных ответов 0.7589424572317263
Лучший параметр С 10


Доля правильных ответов еще немного увеличилась. Она перешла необходимый минимум в 0.75, однако данная модель все еще показывает худшие результаты, чем первые две.

Попробуем также подобрать лучшие гиперпараметры с помощью поиска по сетке.

In [27]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression(random_state=12345)
logreg_cv=GridSearchCV(logreg,grid,cv=10, scoring = 'accuracy')
logreg_cv.fit(features_train_norm,target_train)

print("tuned hyperparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hyperparameters :(best parameters)  {'C': 10.0, 'penalty': 'l1'}
accuracy : 0.7505186721991701


Для логистической регрессии поиск по сетке не улучшил результат.

### Вывод

Модель на алгоритме случайного леса показала самую высокую долю правильных ответов, модель на алгоритме дерева решений попала на второе место, а логистическая регрессия оказалась на третьем месте.

## Проверьте модель на тестовой выборке

Проверим лучшую модель на тестовой выборке.

In [35]:
answer = second_model_cv.predict(features_test)
print('Доля правильных ответов второй модели', accuracy_score(target_test, answer))

Доля правильных ответов второй модели 0.7822706065318819


Доля правильных ответов модели на тестовой выборке немного ниже доли правильных ответов на валидационной выборке.

## (бонус) Проверьте модели на адекватность

Для проверки на адекватность сравним нашу модель со случайной. Получим ее с помощью алгоритма DummyClassifier из библиотеки sklearn. Посмотрим, каково соотношение классов в в целевом признаке, чтобы выбрать нужную стратегию для генерации случайной модели.

In [31]:
target_train.value_counts(normalize=True)

0    0.694502
1    0.305498
Name: is_ultra, dtype: float64

Соотношение классов очень неравное. Зададим параметр strategy='most_frequent', что значит, что ответ будет состоять только из преобладающего в целевом признаке класса.

In [32]:
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_clf.fit(features_train, target_train)
dummy_answer = dummy_clf.predict(features_test)
dummy_accuracy = accuracy_score(target_test, dummy_answer)
print('Доля правильных ответов случайной модели', dummy_accuracy)

Доля правильных ответов случайной модели 0.6889580093312597


Сравним наши модели со случайной.

In [36]:
for i in [first_model_cv.best_score_, second_model_cv.best_score_, third_accuracy]:
    if i < dummy_accuracy:
        print('Модель неадекватна!')
    else:
        print('Модель адекватна')

Модель адекватна
Модель адекватна
Модель адекватна


Все три модели прошли проверку на адекватность.

## Вывод

Мы разбили данные на три выборки - обучающую, валидационную и тестовую. На них проверили модели обученные на трех алгоритмах - дерево решений, случайный лес и логистическая регрессия.

Для алгоритма дерево решений мы подобрали лучшие варианты для следующих гиперпараметров - максимальной глубины (6), минимального количества образцов для разделения внутреннего узла (22) и критерия оценки качества разделения (entropy).

Для алгоритма случайный лес мы подобрали лучшие варианты для следующих гиперпараметров - максимальной глубины (9), количества деревьев (81).

Для алгоритма случайная регрессия мы подобрали лучший вариант параметра С (10) и нормализовали данные для того чтобы увеличить точность.

После подбора наилучших гиперпараметров лучшие показатели доли правильных ответов показали первая и вторая модель. Мы проверили их на тестовой выборке и дерево решений показало лучший результат. 

Дополнительно сделали проверку на адекватность всех моделей. Все они прошли проверку.

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
